In [0]:
%run ../Workspace/Shared/config/config

In [0]:

from pyspark.sql.functions import current_timestamp, lit, current_date

In [0]:
caminho_raw = f"abfss://raw@storagedatanexus.dfs.core.windows.net/"
caminho_bronze = f"abfss://bronze@storagedatanexus.dfs.core.windows.net/"
checkpoint_balanca = f"abfss://checkpoint@storagedatanexus.dfs.core.windows.net/bronze/balancacomercial/"
schema_balanca = f"abfss://checkpoint@storagedatanexus.dfs.core.windows.net/schema/balancacomercial/"
checkpoint_cnpj = f"abfss://checkpoint@storagedatanexus.dfs.core.windows.net/bronze/cnpj/"
schema_cnpj = f"abfss://checkpoint@storagedatanexus.dfs.core.windows.net/schema/cnpj/"

dominios = {
    'Empresas': 'Empresas*',
    'Estabelecimentos': 'Estabelecimentos*',
    'Socios': 'Socios*',
    'Cnaes': 'Cnaes*',
    'Motivos': 'Motivos*',
    'Municipios': 'Municipios*',
    'Paises': 'Paises*',
    'Qualificacoes': 'Qualificacoes*',
    'Simples': 'Simples*',
    'Naturezas': 'Naturezas*'
 }

In [0]:
# INGESTÃO DA RAW -> BRONZE JUNTANDO OS DOMINIOS

for dominio, pattern in dominios.items():
    df = (
      spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.schemaLocation", f"{checkpoint_cnpj}/schema/{dominio}")
        .option("cloudFiles.inferColumnTypes", "false")
        .option("header", "false")
        .option("sep", ";")
        .option("encoding", "latin1")
        .load(f"{caminho_raw}cnpj_descompactados/{pattern}*")
    )

    df = (
            df
            .withColumn('data_ingestao', current_date())
            .withColumn('data_ingestao_timestamp', current_timestamp())
            .withColumn('caminho_fonte', lit(caminho_zip_fonte))
        )
    
    (df
        .writeStream
        .format("delta")
        .option("checkpointLocation", f"{checkpoint_cnpj}/checkpoint/{dominio}")
        .trigger(availableNow=True)
        .start(f"{caminho_bronze}cnpj/{dominio}")
    )